# Reproject to 5km resolution

This will provide a small netcdf we can upload to Zenodo for a publishing a new version.

## Import libraries

In [ ]:
import os
import pickle
import xarray as xr
import numpy as np
from odc.geo.geom import Geometry
from odc.geo.xr import assign_crs
import matplotlib.pyplot as plt

import sys
sys.path.append('/g/data/xc0/project/AusEFlux/src/')
from _utils import round_coords

## Start dask client

In [ ]:
from datacube.utils.dask import start_local_dask
client = start_local_dask(mem_safety_margin='2Gb')
client

## Analysis Parameters

In [ ]:
var = 'ET'
end_year='2024'
version='v2.1'

## Open version 1 & 2

In [ ]:
base = f'/g/data/ub8/au/AusEFlux/'
folder = f'{base}{version}/monthly/{var}'
files = [f'{folder}/{i}' for i in os.listdir(folder) if i.endswith(".nc")]
files.sort()
ds_v2 = xr.open_mfdataset(files)
ds_v2 = assign_crs(ds_v2, crs='EPSG:4326')
ds_v2.attrs['nodata'] = np.nan
attrs = ds_v2.attrs

## Reproject


In [ ]:
# Grab a grid to reproject too
gbox_path = f'/g/data/xc0/project/AusEFlux/data/grid_5km'
with open(gbox_path, 'rb') as f:
    gbox = pickle.load(f)

ds_v2 = ds_v2.odc.reproject(how=gbox, resampling='bilinear').compute()
ds_v2 = round_coords(ds_v2)

ds_v2.attrs = attrs
ds_v2

In [ ]:
ds_v2[f'{var}_median'].mean('time').odc.explore()

## Export to disk

In [ ]:
ds_v2.to_netcdf(f'/g/data/xc0/project/AusEFlux/results/AusEFlux_5km/AusEFlux_{var}_2003_{end_year}_5km_quantiles_{version}.nc')

In [ ]:
# import xarray as xr
# from odc.geo.xr import assign_crs
# ds = assign_crs(xr.open_dataset('/g/data/xc0/project/AusEFlux/results/AusEFlux_5km/AusEFlux_ER_2003_2024_5km_quantiles_v2.1.nc'), crs='EPSG:4326')

# ds['ER_median'].sel(time='2003').mean('time').odc.explore( vmin=0, vmax=150)